# Limpeza dos dados
Nesse notebook vamos preparar os dados que utilizaremos ao longo do desenvolvimento do projeto. Os datasets aqui utilizados são importados do portal de dados abertos da UFRN (http://dados.ufrn.br/).

Para esse estudo, analisaremos apenas os dados do curso de Ciência e Tecnologia da UFRN, mais espeficamente a disciplina de Cálculo 2, onde tentaremos prever as possiveis reprovações dos alunos se baseando em seu histórico e na nota da primeira unidade na disciplina.

In [3]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [185]:
# selecione o ano_periodo para análise
ano_periodo = '20151'

In [186]:
matriculas = pd.read_csv(f"data/matricula-de-{ano_periodo}.csv", sep=";")

In [187]:
matriculas.shape

(876468, 11)

In [188]:
matriculas.columns

Index(['id_turma', 'discente', 'id_curso', 'unidade', 'nota', 'reposicao',
       'faltas_unidade', 'media_final', 'numero_total_faltas', 'descricao',
       'Unnamed: 10'],
      dtype='object')

## Seleção dos dados
Vamos selecionar apenas os dados que nos interessam, assim, iniciaremos pegando apenas os dados do curso de CET e depois limitaremos as displinas que nos interessam.

In [189]:
id_cet = 10320810
alunos_cet = matriculas[matriculas.id_curso == id_cet]

In [190]:
alunos_cet.shape

(94943, 11)

### Pegando as turmas
Vamos pegar as turmas dos componentes que nos interessam.

In [191]:
turmas = pd.read_csv(f"data/turmas-{ano_periodo}.csv", sep=";")

In [192]:
turmas.columns

Index(['id_turma', 'codigo_turma', 'id_docente_interno', 'id_docente_externo',
       'observacao', 'id_componente_curricular', 'nivel_ensino',
       'campus_turma', 'local', 'ano', 'periodo', 'data_inicio', 'data_fim',
       'descricao_horario', 'total_solicitacoes', 'capacidade_aluno', 'tipo',
       'distancia', 'data_consolidacao', 'agrupadora', 'id_turma_agrupadora',
       'qtd_aulas_lancadas', 'situacao_turma', 'convenio',
       'modalidade_participantes', 'Unnamed: 25'],
      dtype='object')

In [193]:
turmas.ano.value_counts()

2015    14662
Name: ano, dtype: int64

In [194]:
#turmas['ano_periodo'] = turmas[['ano', 'periodo']].apply(lambda x: x['ano']*10 + x['periodo'], axis=1)
turmas['ano_periodo'] = int(ano_periodo)

In [195]:
turmas.head()

,id_turma,codigo_turma,id_docente_interno,id_docente_externo,observacao,id_componente_curricular,nivel_ensino,campus_turma,local,ano,...,distancia,data_consolidacao,agrupadora,id_turma_agrupadora,qtd_aulas_lancadas,situacao_turma,convenio,modalidade_participantes,Unnamed: 25,ano_periodo
0,1217099,01,18225.0,NaN,NaN,52922,LATO SENSU,NaN,Depto. de Oceanografia e Limnologia,2015,...,f,2015-06-01 12:53:14.625,f,NaN,15.0,CONSOLIDADA,NaN,Presencial,NaN,20151
1,1217100,01,18695.0,NaN,NaN,52921,LATO SENSU,NaN,Depto. de Oceanografia e Limnologia,2015,...,f,2015-10-08 16:19:21.901,f,NaN,8.0,CONSOLIDADA,NaN,Presencial,NaN,20151
2,1217101,01,5753068.0,NaN,NaN,52925,LATO SENSU,NaN,Depto. de Oceanografia e Limnologia,2015,...,f,2015-06-01 12:50:38.962,f,NaN,30.0,CONSOLIDADA,NaN,Presencial,NaN,20151
3,1217102,01,NaN,109565716.0,NaN,52920,LATO SENSU,NaN,Depto. de Oceanografia e Limnologia,2015,...,f,2015-08-26 11:48:34.146,f,NaN,24.0,CONSOLIDADA,NaN,Presencial,NaN,20151
4,1217102,01,NaN,109565745.0,NaN,52920,LATO SENSU,NaN,Depto. de Oceanografia e Limnologia,2015,...,f,2015-08-26 11:48:34.146,f,NaN,24.0,CONSOLIDADA,NaN,Presencial,NaN,20151


In [196]:
columns_sel = ['id_turma', 'id_componente_curricular', 'id_docente_interno',
       'ano_periodo', 'situacao_turma']
turmas = turmas[columns_sel]

In [197]:
turmas.ano_periodo.unique()

array([20151])

In [198]:
id_turmas_cet = alunos_cet.id_turma.unique()

In [199]:
turmas.shape

(14662, 5)

In [200]:
turmas_cet = turmas[turmas.id_turma.isin(id_turmas_cet)]

In [201]:
turmas_cet.shape

(1017, 5)

In [202]:
# turmas_cet[turmas_cet.codigo.str.startswith("ECT")].codigo.unique()

### Relacionando com os componentes curriculares

In [203]:
comp_curriculares = pd.read_csv("data/componentes-curriculares-presenciais-ok.csv", sep=",")

In [204]:
cc_ect = comp_curriculares[comp_curriculares.unidade_responsavel == 'ESCOLA DE CIÊNCIAS E TECNOLOGIA']

In [205]:
disciplinas_desejadas_nome = ['PRE-CÁLCULO', 'VETORES E GEOMETRIA ANALÍTICA', 'CÁLCULO I', 'CÁLCULO II']

disciplinas_desejadas_codigo = ['ECT2101', 'ECT2102', 'ECT2103', 'ECT2201']

id_componentes_desejados = ['57586', '2050800',          # pré-cálculo
                            '48582', '57587', '2050801', # cálculo 1
                            '48584', '57588', '2051052'] # cálculo 2
                            #'2051005'] # VGA

In [206]:
#cc_ect[cc_ect.nome.str.startswith("VET")]

In [207]:
cc_ect[cc_ect.codigo.isin(disciplinas_desejadas_codigo)]

,Unnamed: 0,id_componente,tipo_componente,codigo,nivel,nome,unidade_responsavel,ch_teorico,ch_pratico,ch_estagio,...,ementa,bibliografia,objetivos,conteudo,competencias_habilidades,referencias,ano_programa,periodo_programa,modalidade,curso_componente
22803,22803,57586,MÓDULO,ECT2101,G,PRÉ-CÁLCULO,ESCOLA DE CIÊNCIAS E TECNOLOGIA,60,0,0,...,Números reais. Operações aritméticas. Números ...,NaN,&#8722; Revisar e proporcionar maior compreens...,1. Números reais1.1 Representação fracionária ...,Desenvolver no aluno a habilidade de entender ...,NaN,2014.0,2.0,Presencial,CIÊNCIAS E TECNOLOGIA
22804,22804,57587,MÓDULO,ECT2103,G,CÁLCULO I,ESCOLA DE CIÊNCIAS E TECNOLOGIA,60,0,0,...,Limites e continuidade. Derivada. Aplicações d...,NaN,- Introduzir e capacitar o aluno a entender e ...,1. Limites1.1 Definição. Limites laterais.1.2 ...,Capacitar o aluno a entender e utilizar adequa...,NaN,2016.0,2.0,Presencial,CIÊNCIAS E TECNOLOGIA
22805,22805,57588,MÓDULO,ECT2201,G,CÁLCULO II,ESCOLA DE CIÊNCIAS E TECNOLOGIA,60,0,0,...,Integral definida. Aplicações da integral. Fun...,NaN,- Introduzir o conceito de integral definida b...,1. Integral Definida1.1 Definição e interpreta...,Possibilitar que o aluno tenha domínio dos con...,NaN,2014.0,2.0,Presencial,CIÊNCIAS E TECNOLOGIA
34962,34962,2051005,MÓDULO,ECT2102,G,VETORES E GEOMETRIA ANALÍTICA,ESCOLA DE CIÊNCIAS E TECNOLOGIA,60,0,0,...,Matrizes e Determinantes. Sistemas Lineares. V...,NaN,&#8722; Apresentar métodos de resolução de sis...,1. Matrizes e Sistemas Lineares1.1 Matrizes1.1...,Desenvolver no aluno a habilidade de identific...,NaN,2014.0,2.0,Presencial,CIÊNCIAS E TECNOLOGIA


In [208]:
turmas_cet = turmas_cet[turmas_cet.id_componente_curricular.isin(id_componentes_desejados)]

In [209]:
turmas_cet.shape

(22, 5)

In [210]:
turmas_cet.situacao_turma.value_counts()

CONSOLIDADA    22
Name: situacao_turma, dtype: int64

In [211]:
turmas_cet.id_componente_curricular.value_counts()

57587    8
57586    7
57588    6
48584    1
Name: id_componente_curricular, dtype: int64

### Pegando apenas as matriculas nas turmas que interessam
Hora de filtrar apenas as matriculas nas turmas dos componentes curriculares que utilizaremos.

In [212]:
turmas_ect_selecionadas = turmas_cet.id_turma.unique()

In [213]:
matriculas_selecionadas =  matriculas[matriculas.id_turma.isin(turmas_ect_selecionadas)]

In [214]:
matriculas_selecionadas.columns

Index(['id_turma', 'discente', 'id_curso', 'unidade', 'nota', 'reposicao',
       'faltas_unidade', 'media_final', 'numero_total_faltas', 'descricao',
       'Unnamed: 10'],
      dtype='object')

Removendo os status indesejados

In [215]:
matriculas_selecionadas.descricao.value_counts()

REPROVADO                           2382
EXCLUIDA                            1813
APROVADO POR NOTA                   1296
CANCELADO                           1186
REPROVADO POR MÉDIA E POR FALTAS     831
APROVADO                             735
INDEFERIDO                           534
TRANCADO                             310
DESISTENCIA                           55
REPROVADO POR NOTA                     3
Name: descricao, dtype: int64

In [216]:
status_indesejados = ['INDEFERIDO', 'EXCLUIDA', 'TRANCADO', 'DESISTENCIA', 'CANCELADO']

In [217]:
matriculas_selecionadas = matriculas_selecionadas[~matriculas_selecionadas.descricao.isin(status_indesejados)]

## Salvando os datasets

In [218]:
matriculas_selecionadas.shape

(5247, 11)

In [219]:
matriculas_selecionadas.to_csv(f"data/matriculas_pos_limpeza_{ano_periodo}.csv", sep=";", index=False)

In [220]:
turmas_cet.to_csv(f"data/turmas_pos_limpeza_{ano_periodo}.csv", sep=";", index=False)